In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
project_root = os.path.abspath("")
if project_root not in sys.path:
    sys.path.append(project_root)
import pandas as pd
import numpy as np 
import utils.classification_utils
from utils.classification_utils import *
from pathlib import Path
import harp
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import aeon.io.video as video
from ipywidgets import widgets
from IPython.display import display
import re
import os
import zoneinfo
from datetime import datetime, timezone
from functools import reduce
from src.processing import detect_settings
import src.processing.detect_stage as detect_stage_module

%matplotlib widget

In [2]:
# Multi Date or SubjID Analysis. Can analyze all Sessions for given SubjID (run on subjid only), all SubjIDs for a date (run on date only), or specific SubjID(s) and Date(s) (run on lists of each)
# To analyze all subjids for a date, or vice versa, set the other argument to None
# For Dates: use lists for specific dates [YYYYMMDD], or use range(start_date, end_date) for a date range (inclusive)

subjids = [26]
dates = [20250811, 20250812, 20250813, 20250814, 20250815, 20250818, 20250819, 20250820, 20250821, 20250825, 20250826, 20250827, 20250828, 20250829, 20250901, 20250902, 20250903, 20250904, 20250905, 20250907, 20250908, 20250909, 20250910, 20250912, 20250915, 20250916, 20250917, 20250918, 20250922, 20250923, 20250929, 20251002, 20251003, 20251006, 20251007, 20251008, 20251009, 20251010]

multi_run_results = batch_analyze_sessions(subjids=subjids, dates=dates, save=True, verbose=False, print_summary=True)


[batch_analyze_sessions] Analyzing subject 26, date 20250811...
Detected stage: QuintupleA_Stage10
No hidden rule index found

SUMMARY: TRIAL CLASSIFICATION AND POKE TIME ANALYSIS FOR SUBJECT [26] DATE [20250811]

Sample offset time: 200.0 ms
Minimum sampling time: 350.0 ms
Response time window: 99999.00 s

TRIAL CLASSIFICATIONs:
Hidden Rule Location: Position None (index None)

Hidden Rule Odors: None

Total attempts: 235
-- Non-initiated sequences (total): 47 (20.0%)
    -- Position 1 attempts within trials 0 (0.0%)
    -- Baseline non-initiated sequences 47 (100.0%)
-- Initiated sequences (trials]): 188 (80.0%)

INITIATED TRIALS BREAKDOWN:
-- Completed sequences: 124 ( 66.0%)
   -- Hidden Rule trials (HR): 0 (0.0%)
   -- Hidden Rule Missed (HR_missed): 0 (0.0%)
-- Aborted sequences: 64 ( 34.0%)
   -- Aborted Hidden Rule trials (HR): 0 (0.0%)

REWARDED TRIALS BREAKDOWN:
-- Rewarded: 120 (96.8%)
-- Unrewarded: 4 (3.2%)
-- Reward Timeout: 0 (0.0%)

POKE TIME RANGES BY POSITION:
------

/Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/utils/classification_utils.py:3734: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged[key] = pd.concat(parts, axis=0, ignore_index=True, sort=False) if parts else pd.DataFrame()



SUMMARY: TRIAL CLASSIFICATION AND POKE TIME ANALYSIS FOR SUBJECT [26] DATE [20250812]

Sample offset time: 200.0 ms
Minimum sampling time: 350.0 ms
Response time window: 5.00 s

TRIAL CLASSIFICATIONs:
Hidden Rule Location: Position 1 (index 0)

Hidden Rule Odors: OdorA, OdorB

Total attempts: 216
-- Non-initiated sequences (total): 42 (19.4%)
    -- Position 1 attempts within trials 1 (2.4%)
    -- Baseline non-initiated sequences 41 (97.6%)
-- Initiated sequences (trials]): 174 (80.6%)

INITIATED TRIALS BREAKDOWN:
-- Completed sequences: 127 ( 73.0%)
   -- Hidden Rule trials (HR): 6 (4.7%)
   -- Hidden Rule Missed (HR_missed): 4 (3.1%)
-- Aborted sequences: 47 ( 27.0%)
   -- Aborted Hidden Rule trials (HR): 4 (8.5%)

REWARDED TRIALS BREAKDOWN:
-- Rewarded: 104 (81.9%)
-- Unrewarded: 10 (7.9%)
-- Reward Timeout: 13 (10.2%)

POKE TIME RANGES BY POSITION:
----------------------------------------
Position 1: 380.5 - 1018.0ms (avg: 926.3ms, n=127)
Position 2: 104.4 - 1017.0ms (avg: 962.5m

/Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/utils/classification_utils.py:3734: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged[key] = pd.concat(parts, axis=0, ignore_index=True, sort=False) if parts else pd.DataFrame()



SUMMARY: TRIAL CLASSIFICATION AND POKE TIME ANALYSIS FOR SUBJECT [26] DATE [20250907]

Sample offset time: 200.0 ms
Minimum sampling time: 200.0 ms
Response time window: 2.00 s

TRIAL CLASSIFICATIONs:
Hidden Rule Location: Position 1 (index 0)

Hidden Rule Odors: OdorA, OdorB

Total attempts: 355
-- Non-initiated sequences (total): 102 (28.7%)
    -- Position 1 attempts within trials 32 (31.4%)
    -- Baseline non-initiated sequences 70 (68.6%)
-- Initiated sequences (trials]): 253 (71.3%)

INITIATED TRIALS BREAKDOWN:
-- Completed sequences: 164 ( 64.8%)
   -- Hidden Rule trials (HR): 11 (6.7%)
   -- Hidden Rule Missed (HR_missed): 5 (3.0%)
-- Aborted sequences: 89 ( 35.2%)
   -- Aborted Hidden Rule trials (HR): 9 (10.1%)

REWARDED TRIALS BREAKDOWN:
-- Rewarded: 120 (73.2%)
-- Unrewarded: 5 (3.0%)
-- Reward Timeout: 39 (23.8%)

POKE TIME RANGES BY POSITION:
----------------------------------------
Position 1: 126.0 - 1017.0ms (avg: 521.0ms, n=164)
Position 2: 138.8 - 1018.0ms (avg: 76

...


SUMMARY: TRIAL CLASSIFICATION AND POKE TIME ANALYSIS FOR SUBJECT [26] DATE [20251007]

Sample offset time: 200.0 ms
Minimum sampling time: 350.0 ms
Response time window: 2.75 s

TRIAL CLASSIFICATIONs:
Hidden Rule Location: Position 1 (index 0)

Hidden Rule Odors: OdorC, OdorF

Total attempts: 305
-- Non-initiated sequences (total): 66 (21.6%)
    -- Position 1 attempts within trials 24 (36.4%)
    -- Baseline non-initiated sequences 42 (63.6%)
-- Initiated sequences (trials]): 239 (78.4%)

INITIATED TRIALS BREAKDOWN:
-- Completed sequences: 170 ( 71.1%)
   -- Hidden Rule trials (HR): 0 (0.0%)
   -- Hidden Rule Missed (HR_missed): 170 (100.0%)
-- Aborted sequences: 69 ( 28.9%)
   -- Aborted Hidden Rule trials (HR): 69 (100.0%)

REWARDED TRIALS BREAKDOWN:
-- Rewarded: 144 (84.7%)
-- Unrewarded: 0 (0.0%)
-- Reward Timeout: 26 (15.3%)

POKE TIME RANGES BY POSITION:
----------------------------------------
Position 1: 313.4 - 1019.0ms (avg: 875.7ms, n=170)
Position 2: 64.6 - 1030.0ms (avg:

/Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/utils/classification_utils.py:3734: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged[key] = pd.concat(parts, axis=0, ignore_index=True, sort=False) if parts else pd.DataFrame()



SUMMARY: TRIAL CLASSIFICATION AND POKE TIME ANALYSIS FOR SUBJECT [26] DATE [20251009]

Sample offset time: 200.0 ms
Minimum sampling time: 350.0 ms
Response time window: 2.75 s

TRIAL CLASSIFICATIONs:
Hidden Rule Location: Position 1 (index 0)

Hidden Rule Odors: OdorC, OdorF

Total attempts: 161
-- Non-initiated sequences (total): 17 (10.6%)
    -- Position 1 attempts within trials 7 (41.2%)
    -- Baseline non-initiated sequences 10 (58.8%)
-- Initiated sequences (trials]): 144 (89.4%)

INITIATED TRIALS BREAKDOWN:
-- Completed sequences: 111 ( 77.1%)
   -- Hidden Rule trials (HR): 0 (0.0%)
   -- Hidden Rule Missed (HR_missed): 111 (100.0%)
-- Aborted sequences: 33 ( 22.9%)
   -- Aborted Hidden Rule trials (HR): 32 (97.0%)

REWARDED TRIALS BREAKDOWN:
-- Rewarded: 107 (96.4%)
-- Unrewarded: 3 (2.7%)
-- Reward Timeout: 1 (0.9%)

POKE TIME RANGES BY POSITION:
----------------------------------------
Position 1: 35.8 - 1025.0ms (avg: 890.1ms, n=111)
Position 2: 95.4 - 1021.0ms (avg: 889.

/Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/utils/classification_utils.py:3734: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged[key] = pd.concat(parts, axis=0, ignore_index=True, sort=False) if parts else pd.DataFrame()



SUMMARY: TRIAL CLASSIFICATION AND POKE TIME ANALYSIS FOR SUBJECT [26] DATE [20251010]

Sample offset time: 200.0 ms
Minimum sampling time: 350.0 ms
Response time window: 2.75 s

TRIAL CLASSIFICATIONs:
Hidden Rule Location: Position 1 (index 0)

Hidden Rule Odors: OdorC, OdorF

Total attempts: 203
-- Non-initiated sequences (total): 32 (15.8%)
    -- Position 1 attempts within trials 10 (31.2%)
    -- Baseline non-initiated sequences 22 (68.8%)
-- Initiated sequences (trials]): 171 (84.2%)

INITIATED TRIALS BREAKDOWN:
-- Completed sequences: 128 ( 74.9%)
   -- Hidden Rule trials (HR): 1 (0.8%)
   -- Hidden Rule Missed (HR_missed): 126 (98.4%)
-- Aborted sequences: 43 ( 25.1%)
   -- Aborted Hidden Rule trials (HR): 43 (100.0%)

REWARDED TRIALS BREAKDOWN:
-- Rewarded: 122 (95.3%)
-- Unrewarded: 2 (1.6%)
-- Reward Timeout: 4 (3.1%)

POKE TIME RANGES BY POSITION:
----------------------------------------
Position 1: 247.9 - 1028.0ms (avg: 873.6ms, n=128)
Position 2: 111.2 - 1029.0ms (avg: 8

In [32]:
res = analyze_session_multi_run_by_id_date(26, 20251009, verbose=False, print_summary=True, save=False)

Detected stage: HiddenRuleA_Location0
Hidden rule index: 0
Detected stage: HiddenRuleA_Location0
Hidden rule index: 0
Detected stage: HiddenRuleA_Location0
Hidden rule index: 0
Detected stage: HiddenRuleA_Location0
Hidden rule index: 0
Detected stage: HiddenRuleA_Location0
Hidden rule index: 0
Detected stage: HiddenRuleA_Location0
Hidden rule index: 0
Detected stage: HiddenRuleA_Location0
Hidden rule index: 0
Detected stage: HiddenRuleA_Location0
Hidden rule index: 0
Detected stage: HiddenRuleA_Location0
Hidden rule index: 0

SUMMARY: TRIAL CLASSIFICATION AND POKE TIME ANALYSIS FOR SUBJECT [26] DATE [20251009]

Sample offset time: 200.0 ms
Minimum sampling time: 350.0 ms
Response time window: 2.75 s

TRIAL CLASSIFICATIONs:
Hidden Rule Location: Position 1 (index 0)

Hidden Rule Odors: OdorC, OdorF

Total attempts: 161
-- Non-initiated sequences (total): 17 (10.6%)
    -- Position 1 attempts within trials 7 (41.2%)
    -- Baseline non-initiated sequences 10 (58.8%)
-- Initiated sequence

/Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/utils/classification_utils.py:3723: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  if isinstance(df, pd.DataFrame) and not df.empty:


In [34]:
# Loading experiments --> just define the SUBJID and DATE
root = load_experiment(38, 20251017, index=0) #can add index for multiple experiments; index=0 as default
stage = detect_stage_module.detect_stage(root)

Using subject directory: /Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/data/rawdata/sub-038_id-235
Loaded experiment 0: /Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/data/rawdata/sub-038_id-235/ses-002_date-20251017/behav/2025-10-17T11-57-05
Detected stage: OdourDiscriminationA_Stage2
No hidden rule index found


In [16]:
root = load_experiment(26, 20250929, index=0)
time_window = ('15:46:30', '15:47:30')
plot_valve_activity = plot_valve_and_poke_events(root=root, time_window=None)

Using subject directory: /Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/data/rawdata/sub-026_id-077
Loaded experiment 0: /Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/data/rawdata/sub-026_id-077/ses-132_date-20250929/behav/2025-09-29T11-24-11
Found 15 experiment files in /Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/data/rawdata/sub-026_id-077/ses-132_date-20250929/behav


KeyboardInterrupt: 

In [35]:
data = load_all_streams(root)

Loading data streams from: /Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/data/rawdata/sub-038_id-235/ses-002_date-20251017/behav/2025-10-17T11-57-05
Loaded heartbeat data
Calculated real-time offset: 44464 days, 17:22:43
Created timestamp interpolation mapping
Loaded digital_input_data
Loaded output_set
Loaded output_clear
Loaded olfactometer_valves_0
Loaded olfactometer_valves_1
Loaded olfactometer_end_0
Loaded analog_data
Loaded flow_meter
Loaded video_data
Loaded pulse_supply_1
Loaded pulse_supply_2
Created odour_led

Applying time corrections to all data streams...
Applied correction to digital_input_data
Applied correction to output_set
Applied correction to output_clear
Applied correction to olfactometer_valves_0
Applied correction to olfactometer_valves_1
Applied correction to olfactometer_end_0
Applied correction to analog_data
Applied correction to flow_meter
Applied correction to video_data
Applied correction to pulse_supply_1
Applied correction to pulse_supply_2
A

In [36]:
events = load_experiment_events(root)

Loading experiment events...
Loaded heartbeat data for timing synchronization
Calculated real-time offset: 44464 days, 17:22:43
Created timestamp interpolation mapping
Found 3 experiment event files
Error processing event file ExperimentEvents_1904-01-21T20-00-00.csv: No columns to parse from file
Processing event file: ExperimentEvents_1904-01-21T19-00-00.csv with 77 rows
Using Seconds column with interpolation
Applied real-time offset: 44464 days, 17:22:43
Found Value column with values: ['EndInitiation' 'ActivateReward' 'AwaitReward' 'Reset'
 'ChooseRandomSequence' 'SampleRewardCondition' 'InitiationSequence']
Found 11 EndInitiation events
Found 11 InitiationSequence events
Found 11 Reset events
Found 11 AwaitReward events
Found 11 SampleRewardCondition events
Found 11 ChooseRandomSequence events
Error processing event file ._ExperimentEvents_1904-01-21T19-00-00.csv: 'utf-8' codec can't decode byte 0xb0 in position 37: invalid start byte
Combined 11 InitiationSequence events
Combine

In [31]:
odor_map = load_odor_mapping(root, data=data, verbose=True)
print("Data streams loaded:", list(data.keys()))
print("Event types loaded:", list(events.keys()))
print("Odor mapping:", odor_map['odour_to_olfactometer_map'])

Loading odor mapping from session settings...
Loaded session settings
Found 10 valve configurations for olfactometer 0
Found 10 valve configurations for olfactometer 1
Created valve-to-command mapping: {'00': 3, '01': 7, '02': 5, '03': 6, '10': 4, '11': 8, '12': 9, '13': 2}
Created odor mapping: [['OdorA', 'OdorE', 'OdorC', 'OdorD'], ['OdorB', 'OdorF', 'OdorG', 'Purge']]
Odor mapping loaded successfully
Data streams loaded: ['heartbeat', 'real_time_offset', 'timestamp_to_time', 'interpolate_time', 'digital_input_data', 'output_set', 'output_clear', 'olfactometer_valves_0', 'olfactometer_valves_1', 'olfactometer_end_0', 'analog_data', 'flow_meter', 'video_reader', 'video_data', 'pulse_supply_1', 'pulse_supply_2', 'odour_led', 'behavior_reader', 'olfactometer_reader']
Event types loaded: ['combined_initiation_sequence_df', 'combined_end_initiation_df', 'combined_await_reward_df', 'combined_reset_df', 'combined_choose_random_sequence_df', 'combined_sample_reward_condition_df']
Odor mappin

In [23]:
trial_counts = detect_trials(data, events, root, verbose=True)

TRIAL DETECTION
Parameters: minimum_sampling_time=50.0ms, sample_offset_time=200.0ms

InitiationSequence 0: 2025-10-17 13:01:26.362976
  Found 2 poke periods
    Starting attempt 1 at 2025-10-17 13:02:55.859136
      Poke 1: 96.4ms (total: 96.4ms)
      SUCCESS: 96.4ms continuous poke (≥50.0ms)

InitiationSequence 1: 2025-10-17 13:03:25.713984
  Found 1 poke periods
    Starting attempt 1 at 2025-10-17 13:04:30.544256
      Poke 1: 378.0ms (total: 378.0ms)
      SUCCESS: 378.0ms continuous poke (≥50.0ms)

InitiationSequence 2: 2025-10-17 13:04:50.926976
  Found 3 poke periods
    Starting attempt 1 at 2025-10-17 13:05:34.541568
      Poke 1: 488.6ms (total: 488.6ms)
      SUCCESS: 488.6ms continuous poke (≥50.0ms)

InitiationSequence 3: 2025-10-17 13:07:08
  Found 1 poke periods
    Starting attempt 1 at 2025-10-17 13:08:19.889312
      Poke 1: 177.6ms (total: 177.6ms)
      SUCCESS: 177.6ms continuous poke (≥50.0ms)

InitiationSequence 4: 2025-10-17 13:08:40.276992
  Found 8 poke peri

In [24]:
trial_outcomes_complete = classify_and_analyze_with_response_times(data, events, trial_counts, odor_map, stage, root)

CLASSIFYING TRIAL OUTCOMES WITH HIDDEN RULE AND VALVE/POKE TIME ANALYSIS
Sample offset time: 200.0 ms
Minimum sampling time: 50.0 ms
Response time window: 99999.0 s
No Hidden Rule Location found in sequence name: OdourDiscriminationA_Stage2. Proceeding without Hidden Rule analysis.
Found 11 total valve activations (excluding Purge)
Analyzing 10 initiated trials...
Found 11 AwaitReward events
Found 11 total supply port activities

TRIAL CLASSIFICATION RESULTS WITH HIDDEN RULE AND VALVE/POKE TIME ANALYSIS:
Hidden Rule Location: Position None (index None)

Hidden Rule Odors: None

Total attempts: 11
-- Non-initiated sequences (total): 1 (9.1%)
-- Initiated sequences (trials): 10 (90.9%)

INITIATED TRIALS BREAKDOWN:
-- Completed sequences: 10 (100.0%)
   -- Hidden Rule trials (HR): 0 (0.0%)
   -- Hidden Rule Missed (HR_missed): 0 (0.0%)
-- Aborted sequences: 0 (0.0%)
   -- Aborted Hidden Rule trials (HR): 0 (0.0%)

REWARD STATUS BREAKDOWN:
-- Rewarded: 10 (100.0%)
-- Unrewarded: 0 (0.0%)
-

# Miscellaneous 

In [ ]:
cls = res['classification']
# Code like this can be used to extract data from res. This will display all trials with < threshold poke times in completed sequences
comp = cls["completed_sequences_with_response_times"]
pos_pokes_all = build_position_pokes_table(cls)
short_pokes = build_position_pokes_table(cls, threshold_ms=200)
display(short_pokes) 

In [21]:
# Find X s window with the most rewarded trials - used to find video segments

def find_peak_rewarded_window(res, window_sec=40):
    # Get rewarded trials table
    cls = res.get("classification", res)
    df = cls.get("completed_sequence_rewarded", pd.DataFrame())
    if df.empty:
        print("No rewarded trials found.")
        return None

    # Use valve_open_ts as trial time (or poke_first_in if you prefer)
    times = pd.to_datetime(df["sequence_start"], errors="coerce")
    df = df.assign(trial_time=times)
    df = df.dropna(subset=["trial_time"]).sort_values("trial_time").reset_index(drop=True)

    # Find the window with the most rewarded trials
    best_count = 0
    best_start = None
    best_end = None
    best_indices = []

    trial_times = df["trial_time"].values
    n = len(trial_times)
    for i in range(n):
        start = trial_times[i]
        end = start + np.timedelta64(window_sec, "s")
        # Find all trials within [start, end)
        mask = (trial_times >= start) & (trial_times < end)
        count = mask.sum()
        if count > best_count:
            best_count = count
            best_start = start
            best_end = end
            best_indices = np.where(mask)[0]

    print(f"Max rewarded trials in any {window_sec}s window: {best_count}")
    print(f"Window: {best_start} to {best_end}")
    # Optionally display the trials in that window
    display(df.iloc[best_indices])
    return df.iloc[best_indices]

# Example usage:
peak_trials = find_peak_rewarded_window(res, window_sec=40)

Max rewarded trials in any 40s window: 3
Window: 2025-09-08T13:56:21.521888000 to 2025-09-08T13:57:01.521888000


,initiation_sequence_time,sequence_start,sequence_end,continuous_poke_time_ms,trial_id,attempt_number,timestamp,odor_sequence,num_odors,last_odor,...,hit_hidden_rule,await_reward_time,first_supply_time,first_supply_port,first_supply_odor_identity,supply1_count,supply2_count,total_supply_count,run_id,trial_time
0,2025-09-08 13:56:20.396000,2025-09-08 13:56:21.521888,2025-09-08 13:56:29.252992,303.456,1,1,2025-09-08 13:56:21.521888,"[OdorE, OdorD, OdorC, OdorF, OdorB]",5,OdorB,...,False,2025-09-08 13:56:26.602976,2025-09-08 13:56:27.742496,2,B,0,1,1,1,2025-09-08 13:56:21.521888
1,2025-09-08 13:56:29.252992,2025-09-08 13:56:37.138528,2025-09-08 13:56:44.724992,447.904,2,1,2025-09-08 13:56:37.138528,"[OdorD, OdorC, OdorF, OdorG, OdorA]",5,OdorA,...,False,2025-09-08 13:56:41.945984,2025-09-08 13:56:43.218496,1,A,1,0,1,1,2025-09-08 13:56:37.138528
2,2025-09-08 13:56:44.724992,2025-09-08 13:56:53.244800,2025-09-08 13:57:01.756096,221.408,3,1,2025-09-08 13:56:53.244800,"[OdorE, OdorG, OdorD, OdorC, OdorA]",5,OdorA,...,False,2025-09-08 13:56:58.558976,2025-09-08 13:57:00.239488,1,A,1,0,1,1,2025-09-08 13:56:53.244800


In [7]:
# Cutting video segments around start and end times. E.g., used to get video around peak rewarded trials identified above. 
# Automatically finds correct file, but runs faster if index is specified
# specify 30 or 60 fps (fps mismatch visible if video duration is not matching input duration)
start_time = "15:03:00"
end_time = "15:07:04"
cut_video(38, 20251027, start_time, end_time, fps=60)

Attempting to read from: /Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/data/rawdata/sub-038_id-235/ses-008_date-20251027/behav/2025-10-27T15-02-39/VideoData/VideoData_1904-01-01T00-00-00.avi
Video has 55705 total frames, requesting frames: 1102 to 15742
Successfully read 14641 frames from /Users/joschua/repos/harris_lab/hypnose/hypnose-analysis/data/rawdata/sub-038_id-235/ses-008_date-20251027/behav/2025-10-27T15-02-39/VideoData/VideoData_1904-01-01T00-00-00.avi
MoviePy - Building video /Volumes/harris/hypnose/derivatives/sub-038_id-235/ses-008_date-20251027/video_analysis/video_cut_15-03-00-000000_15-07-04-000000.mp4.
MoviePy - Writing video /Volumes/harris/hypnose/derivatives/sub-038_id-235/ses-008_date-20251027/video_analysis/video_cut_15-03-00-000000_15-07-04-000000.mp4



MoviePy - Done !
MoviePy - video ready /Volumes/harris/hypnose/derivatives/sub-038_id-235/ses-008_date-20251027/video_analysis/video_cut_15-03-00-000000_15-07-04-000000.mp4
Saved cut video to: /Volumes/harris/hypnose/derivatives/sub-038_id-235/ses-008_date-20251027/video_analysis/video_cut_15-03-00-000000_15-07-04-000000.mp4


PosixPath('/Volumes/harris/hypnose/derivatives/sub-038_id-235/ses-008_date-20251027/video_analysis/video_cut_15-03-00-000000_15-07-04-000000.mp4')

# Testing of new functions

# Debugging Functions:



In [48]:
# Quick code to look for brief purge events in between odors in completed trials

import pandas as pd
import numpy as np
import re
from bisect import bisect_left, bisect_right

def list_short_purge_between_odors(trial_outcomes_complete, odor_map, threshold_ms=200.0, onset_slack_ms=50.0, verbose=True):
    """
    Find Purge events shorter than threshold_ms that occur between distinct odor presentations
    in completed trials. A Purge onset is counted if it lies within:
       [current_odor_end - onset_slack_ms, next_odor_start + onset_slack_ms]
    Returns a list of dicts (one per event) and prints a summary.
    """

    # --- Helpers to resolve Purge columns ---
    def _ci_get(d, key):
        if not isinstance(d, dict):
            return None
        lk = str(key).lower()
        for k, v in d.items():
            if str(k).lower() == lk:
                return v
        return None

    def _col_to_idx(col, row_len=None):
        if isinstance(col, (int, np.integer)):
            idx_num = int(col)
        else:
            m = re.search(r'(\d+)', str(col))
            if not m:
                return None
            idx_num = int(m.group(1))
        if row_len is None:
            return idx_num
        if 0 <= idx_num < row_len:
            return idx_num
        if 1 <= idx_num <= row_len:
            return idx_num - 1
        return None

    def _resolve_odor_name(odor_map, olf_id, idx, col=None):
        v2o = odor_map.get("valve_to_odor", {})
        if isinstance(v2o, dict):
            name = v2o.get((olf_id, idx))
            if name is None and col is not None:
                name = v2o.get(col)
            if name is None:
                name = v2o.get(idx)
            if isinstance(name, str):
                return name
        grid = odor_map.get("odour_to_olfactometer_map") or odor_map.get("odor_to_olfactometer_map")
        if isinstance(grid, (list, tuple)) and len(grid) > olf_id and isinstance(grid[olf_id], (list, tuple)):
            row = grid[olf_id]
            if 0 <= idx < len(row):
                return row[idx]
        return None

    def _purge_columns(odor_map):
        cols = []
        olf_valves = odor_map["olfactometer_valves"]
        grid = odor_map.get("odour_to_olfactometer_map") or odor_map.get("odor_to_olfactometer_map")
        for olf_id, df in olf_valves.items():
            if df is None or getattr(df, "empty", True):
                continue
            row_len = len(grid[olf_id]) if isinstance(grid, (list, tuple)) and len(grid) > olf_id else None
            for col in df.columns:
                idx = _col_to_idx(col, row_len=row_len)
                if idx is None:
                    continue
                name = _resolve_odor_name(odor_map, olf_id, idx, col=col)
                if isinstance(name, str) and name.lower() == "purge":
                    cols.append((olf_id, idx, col))
        return cols

    def _purge_events(odor_map):
        """Return list of dicts: {start,end,duration_ms,olf_id,idx,col} across all purge columns."""
        evs = []
        for olf_id, idx, col in _purge_columns(odor_map):
            df = odor_map["olfactometer_valves"][olf_id]
            if df is None or getattr(df, "empty", True):
                continue
            s = df[col].astype(bool)
            rises = s & ~s.shift(1, fill_value=False)
            falls = ~s & s.shift(1, fill_value=False)
            starts = list(s.index[rises])
            ends = list(s.index[falls])
            i = j = 0
            while i < len(starts) and j < len(ends):
                if ends[j] <= starts[i]:
                    j += 1
                    continue
                dur_ms = (ends[j] - starts[i]).total_seconds() * 1000.0
                evs.append({
                    "start": starts[i],
                    "end": ends[j],
                    "duration_ms": dur_ms,
                    "olf_id": olf_id,
                    "idx": idx,
                    "col": col,
                })
                i += 1
                j += 1
        evs.sort(key=lambda e: e["start"])
        return evs

    # --- Build completed trials dataframe and inter-odor windows ---
    cls = trial_outcomes_complete.get("classification", trial_outcomes_complete)
    completed_keys = [
        "completed_sequence_rewarded",
        "completed_sequence_unrewarded",
        "completed_sequence_reward_timeout",
    ]
    completed_dfs = [cls[k] for k in completed_keys if k in cls and isinstance(cls[k], pd.DataFrame)]
    completed_df = pd.concat(completed_dfs, ignore_index=True) if completed_dfs else pd.DataFrame()

    def _trial_id(row):
        for k in ["trial_id", "trial_index", "sequence_index", "Trial", "Sequence", "trial"]:
            if k in row and pd.notna(row[k]):
                return row[k]
        return row.name

    # Collect inter-odor windows per trial: [(trial_id, pos_i, pos_j, win_start, win_end)]
    windows = []
    for _, row in completed_df.iterrows():
        pov = row.get("position_valve_times")
        if not isinstance(pov, dict) or not pov:
            continue
        tid = _trial_id(row)
        # sort positions by position number
        positions = sorted([p for p in pov.keys() if isinstance(p, (int, np.integer))])
        # windows between consecutive positions
        for i in range(len(positions) - 1):
            p_i = positions[i]
            p_j = positions[i + 1]
            end_i = pov[p_i].get("valve_end")
            start_j = pov[p_j].get("valve_start")
            if pd.isna(end_i) or pd.isna(start_j) or end_i is None or start_j is None:
                continue
            if end_i >= start_j:
                # overlapping/invalid; skip
                continue
            win_start = end_i - pd.Timedelta(milliseconds=onset_slack_ms)
            win_end = start_j + pd.Timedelta(milliseconds=onset_slack_ms)
            windows.append((tid, p_i, p_j, win_start, win_end))

    # --- Scan purge events and match onsets within windows ---
    purge_events = _purge_events(odor_map)
    starts = [e["start"] for e in purge_events]

    matches = []
    for tid, p_i, p_j, ws, we in windows:
        lo = bisect_left(starts, ws)
        hi = bisect_right(starts, we)
        for k in range(lo, hi):
            e = purge_events[k]
            if e["duration_ms"] < threshold_ms:
                matches.append({
                    "trial_id": tid,
                    "from_pos": p_i,
                    "to_pos": p_j,
                    "start": e["start"],
                    "end": e["end"],
                    "duration_ms": e["duration_ms"],
                    "olf_id": e["olf_id"],
                    "col": e["col"],
                })

    # Summary/print
    if verbose:
        print(f"Short Purge events (< {threshold_ms} ms) between odors in completed trials "
              f"(onset slack ±{onset_slack_ms} ms): {len(matches)}")
        by_trial = {}
        for m in matches:
            by_trial.setdefault(m["trial_id"], 0)
            by_trial[m["trial_id"]] += 1
        if matches:
            for m in sorted(matches, key=lambda x: (x["trial_id"], x["start"])):
                print(f"- trial {m['trial_id']} pos {m['from_pos']}->{m['to_pos']}: "
                      f"{m['start'].isoformat()} -> {m['end'].isoformat()} "
                      f"({m['duration_ms']:.1f} ms)  olf {m['olf_id']} col '{m['col']}'")
            print("\nCounts by trial:")
            for tid in sorted(by_trial):
                print(f"  trial {tid}: {by_trial[tid]}")

    return matches

# Example usage:
matches = list_short_purge_between_odors(trial_outcomes_complete, odor_map, threshold_ms=200.0, onset_slack_ms=100.0, verbose=True)
len(matches)

Short Purge events (< 200.0 ms) between odors in completed trials (onset slack ±100.0 ms): 113
- trial 4 pos 3->4: 2025-09-07T14:36:46.469536 -> 2025-09-07T14:36:46.605504 (136.0 ms)  olf 1 col 'Valve3'
- trial 5 pos 4->5: 2025-09-07T14:37:06.771488 -> 2025-09-07T14:37:06.870496 (99.0 ms)  olf 1 col 'Valve3'
- trial 6 pos 2->3: 2025-09-07T14:37:21.450496 -> 2025-09-07T14:37:21.620480 (170.0 ms)  olf 1 col 'Valve3'
- trial 7 pos 2->3: 2025-09-07T14:38:24.026496 -> 2025-09-07T14:38:24.108480 (82.0 ms)  olf 1 col 'Valve3'
- trial 7 pos 3->4: 2025-09-07T14:38:24.811520 -> 2025-09-07T14:38:24.934496 (123.0 ms)  olf 1 col 'Valve3'
- trial 9 pos 1->2: 2025-09-07T14:38:55.848480 -> 2025-09-07T14:38:55.997504 (149.0 ms)  olf 1 col 'Valve3'
- trial 11 pos 2->3: 2025-09-07T14:39:29.279488 -> 2025-09-07T14:39:29.375488 (96.0 ms)  olf 1 col 'Valve3'
- trial 13 pos 1->2: 2025-09-07T14:39:59.003488 -> 2025-09-07T14:39:59.202496 (199.0 ms)  olf 1 col 'Valve3'
- trial 14 pos 3->4: 2025-09-07T14:40:23.7

113

In [ ]:
# For a schema file, check any poke that is < threshold (e.g. 200 ms), within completed sequences
import pandas as pd
import json, ast
from pathlib import Path
from collections.abc import Mapping

def _parse_obj(x):
    if isinstance(x, (dict, list, tuple)):
        return x
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return None
        # Try JSON first, then Python literal
        try:
            return json.loads(s)
        except Exception:
            try:
                return ast.literal_eval(s)
            except Exception:
                return None
    return None

def _iter_pos_items(ppt):
    # Yield (position, info_dict) for dict/list
    if isinstance(ppt, Mapping):
        for k, v in ppt.items():
            if not isinstance(v, Mapping):
                try:
                    v = dict(v)
                except Exception:
                    continue
            pos = v.get("position")
            if pos is None:
                try:
                    pos = int(k)
                except Exception:
                    pos = k
            yield pos, v
    elif isinstance(ppt, (list, tuple)):
        for v in ppt:
            if isinstance(v, Mapping):
                yield v.get("position"), v

def _normalize_valves(pvt):
    # Normalize to dict: position -> valve dict
    out = {}
    if isinstance(pvt, Mapping):
        items = list(pvt.items())
    elif isinstance(pvt, (list, tuple)):
        items = [(v.get("position"), v) for v in pvt if isinstance(v, Mapping)]
    else:
        items = []
    for k, v in items:
        if not isinstance(v, Mapping):
            try:
                v = dict(v)
            except Exception:
                v = {}
        pos = v.get("position")
        if pos is None:
            try:
                pos = int(k)
            except Exception:
                pos = k
        out[pos] = v
    return out

def extract_short_pokes_from_saved(schema_path: str | Path, threshold_ms: float = 200.0) -> pd.DataFrame:
    schema_path = Path(schema_path)
    csv_path = schema_path.with_suffix("").with_suffix(".csv")  # replace .schema.json -> .csv
    if not csv_path.exists():
        # fallback: try sibling CSV with same stem
        csv_path = schema_path.parent / (schema_path.stem.replace(".schema", "") + ".csv")
    if not csv_path.exists():
        raise FileNotFoundError(f"CSV not found next to schema: {csv_path}")

    df = pd.read_csv(csv_path)

    # Identify column names for per-position fields
    poke_col = None
    valve_col = None
    for c in df.columns:
        lc = c.lower()
        if poke_col is None and "position" in lc and "poke" in lc:
            poke_col = c
        if valve_col is None and "position" in lc and "valve" in lc:
            valve_col = c
    if poke_col is None:
        raise KeyError("Could not find position_poke_times column in CSV")
    if valve_col is None:
        # some outputs may not store per-position valves; still proceed
        valve_col = None

    rows = []
    for _, row in df.iterrows():
        ppt = _parse_obj(row.get(poke_col))
        if ppt is None:
            continue
        pvt_raw = _parse_obj(row.get(valve_col)) if valve_col else None
        valve_map = _normalize_valves(pvt_raw) if pvt_raw is not None else {}

        run_id = row.get("run_id")
        trial_id = row.get("trial_id")

        for pos, info in _iter_pos_items(ppt):
            if not isinstance(info, Mapping):
                try:
                    info = dict(info)
                except Exception:
                    continue
            poke_ms = pd.to_numeric(info.get("poke_time_ms"), errors="coerce")
            if pd.isna(poke_ms) or poke_ms <= 0 or poke_ms >= threshold_ms:
                continue

            # normalize pos
            try:
                pos_norm = int(pos) if pos is not None else None
            except Exception:
                pos_norm = pos

            vt = valve_map.get(pos_norm, {})
            odor = info.get("odor_name") or (vt or {}).get("odor_name")
            first_in = info.get("poke_first_in")
            valve_open = (vt or {}).get("valve_open_ts")
            valve_close = (vt or {}).get("valve_close_ts")
            event_ts = first_in if first_in is not None else valve_open  # measurement start

            rows.append({
                "run_id": run_id,
                "trial_id": trial_id,
                "position": pos_norm,
                "odor": odor,
                "poke_ms": float(poke_ms),
                "event_ts": event_ts,
                "valve_open_ts": valve_open,
                "valve_close_ts": valve_close,
                "poke_first_in": first_in,
            })

    out = pd.DataFrame(rows)
    if not out.empty:
        out["event_ts"] = pd.to_datetime(out["event_ts"], errors="coerce")
        out = out.sort_values(["run_id","trial_id","position","event_ts"], kind="stable", na_position="last").reset_index(drop=True)
    return out

# Use your saved path
schema_path = "/Volumes/harris/hypnose/derivatives/sub-026_id-077/ses-122_date-20250909/saved_analysis_results/completed_sequences_with_response_times.schema.json"
short_pokes = extract_short_pokes_from_saved(schema_path, threshold_ms=200.0)
display(short_pokes)
print(f"{len(short_pokes)} positions with poke_time_ms < 200 ms")

In [11]:
# Identify trial IDs for trials with poke time bewlow threshold 
threshold_ms = 200.0

# Get the classification DataFrames
cls = trial_outcomes_complete.get("classification", trial_outcomes_complete)

completed_keys = [
    "completed_sequence_rewarded",
    "completed_sequence_unrewarded",
    "completed_sequence_reward_timeout",
]
completed_dfs = [cls[k] for k in completed_keys if k in cls and isinstance(cls[k], pd.DataFrame)]
completed_df = pd.concat(completed_dfs, ignore_index=True) if completed_dfs else pd.DataFrame()

def _trial_id(row):
    for k in ["trial_id", "trial_index", "sequence_index", "Trial", "Sequence", "trial"]:
        if k in row and pd.notna(row[k]):
            return row[k]
    return row.name  # fallback

hits = []
for _, row in completed_df.iterrows():
    pos_pokes = row.get("position_poke_times")
    if not isinstance(pos_pokes, dict):
        continue
    tid = _trial_id(row)
    for pos, info in pos_pokes.items():
        if not isinstance(info, dict):
            continue
        ms = info.get("poke_time_ms")
        if ms is None:
            continue
        if ms < threshold_ms:
            odor = info.get("odor_name")
            hits.append((ms, tid, pos, odor))

# Print results sorted by poke time
if not hits:
    print(f"No completed-trial positions with poke_time_ms < {threshold_ms} ms found.")
else:
    hits.sort(key=lambda x: x[0])
    print(f"Trials with poke_time_ms < {threshold_ms} ms (n={len(hits)}):")
    for ms, tid, pos, odor in hits:
        print(f"- trial {tid}, position {pos}, {odor}: {ms:.1f} ms")

Trials with poke_time_ms < 200.0 ms (n=11):
- trial 163, position 1, OdorF: 56.8 ms
- trial 107, position 1, OdorG: 100.1 ms
- trial 153, position 2, OdorC: 117.1 ms
- trial 161, position 3, OdorC: 155.9 ms
- trial 51, position 2, OdorE: 171.5 ms
- trial 130, position 2, OdorG: 187.9 ms
- trial 178, position 3, OdorD: 194.8 ms
- trial 179, position 2, OdorF: 197.9 ms
- trial 168, position 1, OdorG: 199.0 ms
- trial 181, position 1, OdorD: 199.8 ms
- trial 11, position 2, OdorE: 199.9 ms
